In [61]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from api_keys import g_key


In [62]:
#Import weather data

weather_df = pd.read_csv("..\\WeatherPy\\weather_data.csv")
weather_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind_Speed,Hemisphere
0,Busselton,-33.65,115.33,52.79,52,0,12.15,Southern
1,Cidreira,-30.18,-50.21,66.56,86,100,19.62,Southern
2,Yellowknife,62.46,-114.35,23.00,85,90,9.17,Northern
3,Ushuaia,-54.80,-68.30,48.20,61,75,21.92,Southern
4,Okakarara,-20.58,17.43,94.57,10,45,11.63,Southern


In [63]:
#Store variables to be used in map

locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

In [64]:
#Create map
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
humidity_heatmap = gmaps.heatmap_layer(locations, weights=humidity, 
                                       dissipating=False, max_intensity=100, 
                                       point_radius = 1)

fig.add_layer(humidity_heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

In [65]:
#Narrow down dataframe

vacation_df = weather_df.loc[(weather_df["Temperature"] < 85) & (weather_df["Temperature"] > 70) & (weather_df["Wind_Speed"] < 10) & (weather_df["Cloudiness"] < 10)]
vacation_df = vacation_df.dropna()
vacation_df.count()

City           30
Latitude       30
Longitude      30
Temperature    30
Humidity       30
Cloudiness     30
Wind_Speed     30
Hemisphere     30
dtype: int64

In [92]:
coords_list = []
for index, row in vacation_df.iterrows():
    coords = str(row['Latitude']) + "," + str(row['Longitude'])
    coords_list.append(coords)

hotel_dict = {"Hotel Names": "", "Locations": coords_list}
hotel_df = pd.DataFrame(hotel_dict)
hotel_df.head()

,Hotel Names,Locations
0,,"-34.42,19.23"
1,,"-6.88,112.21"
2,,"27.53,68.76"
3,,"-6.84,-79.93"
4,,"25.12,62.33"


In [99]:
#Work with Google APIs

hotels_list = []
for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": row["Locations"],
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    response = requests.get(base_url, params)
    hotels = response.json()
    hotels_list.append(hotels)
    

#Put my responses into a list so i didn't have to keep hitting API
hotels_list

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAANpYw2j-TuMXpLPxkbhVX3xl7dJZng0BfmBjyB-Snz3diE1nGLdzeGis3WHxKAcuFNW0YywcTEqdWvVpopYJgjlMqIj8iOsgKgP9gZ4TLNzR0xxW5_VQfrQRJk98RBJRobUJCTF9NoY8q4ImKXcmF_THV4-xV4yA-WuUgbvq1SOGB6_c0-l-56e3pFZ53DelMsWnfePDNK5lHqkTdSMFGKMw04b1c4heebyYRt0nzKazSPRqULdG7sDYceKTTRa39z6nLJWGby2rSDj8jc4AQjg0M3ws6m_jyIq6AM6pB3MHgMO9LH1PrsvvkLQqRH1n1UrY00hJVBVqFqzMSspHwI71-NDoLKTawpHgiHkKBtNiofkY3f6mbOV2E7RoonfhexIQ_S5wPCsk818qkUGNTTmgUBoUFrFEiir4RA5yE2ikksugfdOt9aA',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -20.0065691, 'lng': 57.66807650000001},
    'viewport': {'northeast': {'lat': -20.0053813197085,
      'lng': 57.66935138029151},
     'southwest': {'lat': -20.0080792802915, 'lng': 57.6666534197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Veranda Paul et Virginie Hotel & Spa',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 2592,
     

In [103]:
hotels_list[5]

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAJuCaScCUq4tKReXLCVYq7alxbc4VvcJwjAeEJ4wb-8wUDQ8S0CcD6wypithsQB8Tcu9CXrRGTygJGYkFnSR0xgHvw_HJeNZTdGSyDzXGZxBXXDO8WtWuIGU-uPLpnfSX1xqKqq5vVDCojR6O9WHr7TLAFyuiRMLSy6PGvB0eXeZvhQe3gb0ANgvf2Wgqc9V60ykQARXyVK_yieJJGoh5zHji0lH3WhSw5Hf19bpmHriN7-PCqcoyDAuK1d2CvsbPBOWP2GjCRA9JOOkjtmzYEFmvuEdT5NExFAIF3Om7LvYnPJ618j87GLRMOq9rehBfy_fT2KjX23Vc-GbbLXFlwYiyh5dxhYP46gO8yYMCkvbK7QMlmKfyQ59vLjOuIAcLRIQliZ_N1mt1dScaKwPyBnwxBoUUlV0ISIClhafXQFBtefuEswVsBY',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 17.4899148, 'lng': 44.1288001},
    'viewport': {'northeast': {'lat': 17.4911381802915,
      'lng': 44.1302192802915},
     'southwest': {'lat': 17.4884402197085, 'lng': 44.1275213197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'محمد هادي ال فطيح',
   'place_id': 'ChIJRZMb6fzr_hURILNDHAgDpWs',
   'plus_code': {'compound_code': 'F4QH+XG Najran Saudi Arab

KeyError: 'name'

In [68]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"